In [ ]:
from fastai.gen_doc.gen_notebooks import update_module_metadata
import fastai.models.rnn
# For updating jekyll metadata. You MUST reload notebook immediately after executing this cell for changes to save
# Leave blank to autopopulate from mod.__doc__
update_module_metadata(fastai.models.rnn)

# models.rnn

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.text.models import * 
from fastai import *

This module fully implements the [AWD-LSTM](https://arxiv.org/pdf/1708.02182.pdf) from Stephen Merity et al. The main idea of the article is to use a [RNN](http://www.pnas.org/content/79/8/2554) with dropout everywhere, but in an intelligent way. There is a difference with the usual dropout, which is why you’ll see a `RNNDropout` module: we zero things, as is usual in dropout, but we always zero the same thing according to the sequence dimension (which is the first dimension in pytorch). This ensures consistency when updating the hidden state through the whole sentences/articles. 

This being given, there are five different dropouts in the AWD-LSTM:
- the first one, embedding dropout, is applied when we look the ids of our tokens inside the embedding matrix (to transform them from numbers to a vector of float). We zero some lines of it, so random ids are sent to a vector of zeros instead of being sent to their embedding vector.
- the second one, input dropout, is applied to the result of the embedding with dropout. We forget random pieces of the embedding matrix (but as stated in the last paragraph, the same ones in the sequence dimension).
- the third one is the weight dropout. It’s the trickiest to implement as we randomly replace by 0s some weights of the hidden-to-hidden matrix inside the RNN: this needs to be done in a way that ensure the gradients are still computed and the initial weights still updated.
- the fourth one is the hidden dropout. It’s applied to the output of one of the layers of the RNN before it’s used as input of the next layer (again same coordinates are zeroed in the sequence dimension). This one isn’t applied to the last output, but rather…
- the fifth one is the output dropout, it’s applied to the last output of the model (and like the others, it’s applied the same way through the first dimension).

## Basic functions to get a model

In [ ]:
show_doc(get_language_model, doc_string=False)

#### <a id=get_language_model></a>`get_language_model`
> `get_language_model`(`vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `tie_weights`:`bool`=`True`, `qrnn`:`bool`=`False`, `bias`:`bool`=`True`, `output_p`:`float`=`0.4`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`) -> `Module`
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L205">[source]</a>

Creates an AWD-LSTM with a first embedding of `vocab_sz` by `emb_sz`, a hidden size of `n_hid`, RNNs with `n_layers` that can be bidirectional if `bidir` is True. The last RNN as an output size of `emb_sz` so that we can use the same decoder as the encoder if `tie_weights` is True. The decoder is a `Linear` layer with or without `bias`. If `qrnn` is set to True, we use [QRNN cells] instead of LSTMS. `pad_token` is the token used for padding.

`embed_p` is used for the embedding dropout, `input_p` is used for the input dropout, `weight_p` is used for the weight dropout, `hidden_p` is used for the hidden dropout and `output_p` is used for the output dropout.

Note that the model returns a list of three things, the actual output being the first, the two others being the intermediate hidden states before and after dropout (used by the `RNNTrainer`). Most loss functions expect one output, so you should use a Callback to remove the other two if you're not using `RNNTrainer`.

In [ ]:
show_doc(get_rnn_classifier, doc_string=False)

#### <a id=get_rnn_classifier></a>`get_rnn_classifier`
> `get_rnn_classifier`(`bptt`:`int`, `max_seq`:`int`, `n_class`:`int`, `vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `layers`:`Collection`\[`int`\], `drops`:`Collection`\[`float`\], `bidir`:`bool`=`False`, `qrnn`:`bool`=`False`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`) -> `Module`
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L214">[source]</a>

Creates a RNN classifier with a encoder taken from an AWD-LSTM with arguments `vocab_sz`, `emb_sz`, `n_hid`, `n_layers`, `bias`, `bidir`, `qrnn`, `pad_token` and the dropouts parameters. This encoder is fed the sequence by successive bits of size `bptt` and we only keep the last `max_seq` outputs for the pooling layers.

The decoder use a concatenation of the last outputs, a `MaxPooling` of all the ouputs and an `AveragePooling` of all the outputs. It then uses a list of `BatchNorm`, `Dropout`, `Linear`, `ReLU` blocks (with no `ReLU` in the last one), using a first layer size of `3*emb_sz` then follwoing the numbers in `n_layers` to stop at `n_class`. The dropouts probabilities are read in `drops`.

Note that the model returns a list of three things, the actual output being the first, the two others being the intermediate hidden states before and after dropout (used by the `RNNTrainer`). Most loss functions expect one output, so you should use a Callback to remove the other two if you're not using `RNNTrainer`.

## Basic NLP modules

On top of the pytorch or the fastai `layers`, the language models use some custom layers specific to NLP.

In [ ]:
show_doc(EmbeddingDropout, doc_string=False, title_level=3)

### <a id=EmbeddingDropout></a>`class` `EmbeddingDropout`
> `EmbeddingDropout`(`emb`:`Module`, `embed_p`:`float`) :: `Module`
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L50">[source]</a>

Applies a dropout with probability `embed_p` to an embedding layer `emb` in training mode. Each row of the embedding matrix has a probability `embed_p` of being replaced by zeros while the others are rescaled accordingly. 

In [ ]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

tensor([[ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000, -0.0000],
        [ 0.8444, -0.8950, -0.7440,  0.9375,  1.0407, -0.6192,  0.4821],
        [-1.9462, -0.3018, -0.0422, -1.7010,  0.5663, -0.8133, -0.0367],
        [-0.0000,  0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000],
        [ 3.9847,  3.1350,  2.8408, -3.6548, -1.0186,  1.7506, -1.3603],
        [ 0.0000,  0.0000,  0.0000, -0.0000, -0.0000, -0.0000, -0.0000],
        [-4.0268, -2.4269,  2.3209, -0.3243, -1.6055, -1.3608,  0.4655],
        [-3.4586,  3.0780,  2.9428, -3.8629,  0.6396, -0.8378, -1.0522]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
show_doc(RNNDropout, doc_string=False, title_level=3)

### <a id=RNNDropout></a>`class` `RNNDropout`
> `RNNDropout`(`p`:`float`=`0.5`) :: `Module`
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L11">[source]</a>

Applies a dropout with probability `p` consistently over the first dimension in training mode.

In [ ]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,7)
tst_input, dp(tst_input)

(tensor([[[ 0.3035, -0.5677, -0.3388,  1.2217,  0.3325, -0.4691, -0.0517],
          [ 0.0070,  1.9497, -1.4526,  1.6338, -0.2380,  0.4030, -0.7704],
          [-1.3806,  0.4710, -0.1631, -0.5893, -1.7183, -1.2850, -0.3287]],
 
         [[-1.0775,  0.1940, -0.4856, -0.5217, -1.7239, -0.4365,  0.3934],
          [ 2.4286,  0.2146, -1.0913,  0.4197, -0.1836,  1.9187,  0.3242],
          [-0.1235, -0.4573,  0.0116, -0.8257,  0.9690, -0.1584,  0.9877]],
 
         [[ 0.2453, -0.9624, -2.1222,  0.1307,  1.2131, -0.9540,  1.3581],
          [ 0.1412,  1.2920,  0.2446, -0.4857, -0.0986, -0.4935, -0.8928],
          [ 0.8967,  1.5670,  0.3258,  1.2222, -2.0801,  0.0805, -1.2240]]]),
 tensor([[[ 0.0000, -0.8110, -0.4841,  1.7453,  0.0000, -0.6702, -0.0739],
          [ 0.0000,  2.7852, -0.0000,  0.0000, -0.3400,  0.5757, -1.1005],
          [-0.0000,  0.6728, -0.2330, -0.8418, -2.4546, -1.8357, -0.4695]],
 
         [[-0.0000,  0.2771, -0.6937, -0.7453, -0.0000, -0.6236,  0.5620],
          [ 0

In [ ]:
show_doc(WeightDropout, doc_string=False, title_level=3)

### <a id=WeightDropout></a>`class` `WeightDropout`
> `WeightDropout`(`module`:`Module`, `weight_p`:`float`, `layer_names`:`StrList`=`['weight_hh_l0']`) :: `Module`
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L23">[source]</a>

Applies dropout of probability `weight_p` to the layers in `layer_names` of `module` in training mode. A copy of those weights is kept so that the dropout mask can change at every batch.

In [ ]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, 'weight_hh_l0')

Parameter containing:
tensor([[ 0.2589,  0.4894],
        [ 0.3660, -0.5721],
        [-0.4000, -0.2764],
        [ 0.5950, -0.5937],
        [-0.2155,  0.1092],
        [-0.4131,  0.0684],
        [ 0.5688, -0.6756],
        [ 0.0249,  0.1041]], requires_grad=True)

It's at the beginning of a forward pass that the dropout is applied to the weights.

In [ ]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, 'weight_hh_l0')

tensor([[ 0.0000,  0.8156],
        [ 0.6100, -0.0000],
        [-0.6667, -0.4607],
        [ 0.9917, -0.9894],
        [-0.3592,  0.0000],
        [-0.6884,  0.1139],
        [ 0.9480, -0.0000],
        [ 0.0414,  0.1735]], grad_fn=<MulBackward0>)

In [ ]:
show_doc(SequentialRNN, doc_string=False, title_level=3)

### <a id=SequentialRNN></a>`class` `SequentialRNN`
> `SequentialRNN`(`args`) :: `Sequential`
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L151">[source]</a>

Create a `Sequentiall` module with `args` that has a `reset` function.

In [ ]:
show_doc(SequentialRNN.reset)

#### <a id=reset></a>`reset`
> `reset`()
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L153">[source]</a>

Call the `reset` function of `self.children` (if they have one).

In [ ]:
show_doc(dropout_mask, doc_string=False)

#### <a id=dropout_mask></a>`dropout_mask`
> `dropout_mask`(`x`:`Tensor`, `sz`:`Collection`\[`int`\], `p`:`float`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L7">[source]</a>

Create a dropout mask of size `sz`, the same type as `x` and probability `p`.

In [ ]:
tst_input = torch.randn(3,3,7)
dropout_mask(tst_input, (3,7), 0.3)

tensor([[1.4286, 1.4286, 0.0000, 1.4286, 1.4286, 1.4286, 0.0000],
        [0.0000, 0.0000, 1.4286, 1.4286, 0.0000, 1.4286, 1.4286],
        [0.0000, 1.4286, 1.4286, 0.0000, 1.4286, 1.4286, 1.4286]])

Such a mask is then expanded in the sequence length dimension and multiplied by the input to do an `RNNDropout`.

## Language model modules

In [ ]:
show_doc(RNNCore, doc_string=False, title_level=3)

### <a id=RNNCore></a>`class` `RNNCore`
> `RNNCore`(`vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `bidir`:`bool`=`False`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`, `qrnn`:`bool`=`False`) :: `Module`
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L73">[source]</a>

Create an AWD-LSTM encoder with an embedding layer of `vocab_sz` by `emb_sz`, a hidden size of `n_hid`, `n_layers` layers. `pad_token` is passed to the `Embedding`, if `bidir` is True, the model is bidirectional. If `qrnn` is True, we use QRNN cells instead of LSTMs. Dropouts are `embed_p`, `input_p`, `weight_p` and `hidden_p`.

In [ ]:
show_doc(RNNCore.reset)

#### <a id=reset></a>`reset`
> `reset`()


Reset the hidden states. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L125">[source]</a>

In [ ]:
show_doc(LinearDecoder, doc_string=False, title_level=3)

### <a id=LinearDecoder></a>`class` `LinearDecoder`
> `LinearDecoder`(`n_out`:`int`, `n_hid`:`int`, `output_p`:`float`, `tie_encoder`:`Module`=`None`, `bias`:`bool`=`True`) :: `Module`
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L132">[source]</a>

Create a the decoder to go on top of an `RNNCore` encoder and create a language model. `n_hid` is the dimension of the last hidden state of the encoder, `n_out` the size of the output. Dropout of `output_p` is applied. If a `tie_encoder` is passed, it will be used for the weights of the linear layer, that will have `bias` or not.

## Classifier modules

In [ ]:
show_doc(MultiBatchRNNCore, doc_string=False, title_level=3)

### <a id=MultiBatchRNNCore></a>`class` `MultiBatchRNNCore`
> `MultiBatchRNNCore`(`bptt`:`int`, `max_seq`:`int`, `args`, `kwargs`) :: `RNNCore`
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L157">[source]</a>

Wrap an `RNNCore` to make it process full sentences: text is passed by chunks of sequence length `bptt` and only the last `max_seq` outputs are kept for the next layer. `args` and `kwargs` are passed to the `RNNCore`.

In [ ]:
show_doc(MultiBatchRNNCore.concat)

#### <a id=concat></a>`concat`
> `concat`(`arrs`:`Collection`\[`Tensor`\]) -> `Tensor`


Concatenate the arrays along the batch dimension. <a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L164">[source]</a>

In [ ]:
show_doc(PoolingLinearClassifier, doc_string=False, title_level=3)

### <a id=PoolingLinearClassifier></a>`class` `PoolingLinearClassifier`
> `PoolingLinearClassifier`(`layers`:`Collection`\[`int`\], `drops`:`Collection`\[`float`\]) :: `Module`
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L179">[source]</a>

Create a linear classifier that sits on an `RNNCore` encoder. The last output, `MaxPooling` of all the outputs and `AvgPooling` of all the outputs are concatenated, then blocks of `bn_drop_lin` are stacked, according to the values in `layers` and `drops`.

In [ ]:
show_doc(PoolingLinearClassifier.pool, doc_string=False)

#### <a id=pool></a>`pool`
> `pool`(`x`:`Tensor`, `bs`:`int`, `is_max`:`bool`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/text/models.py#L190">[source]</a>

Pool `x` (of batch size `bs`) along the batch dimension. `is_max` decides if we do an `AvgPooling` or a `MaxPooling`.

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(WeightDropout.forward)

In [ ]:
show_doc(RNNCore.forward)

In [ ]:
show_doc(EmbeddingDropout.forward)

In [ ]:
show_doc(RNNDropout.forward)

## New Methods - Please document or move to the undocumented section